# Fine-Grained Tool Calling

If you need faster, more granular streaming - perhaps to show users immediate updates or start processing partial results quickly you can enable fine-grained tool calling.

### Fine-grained tool calling does one main thing: it disables JSON validation on the API side. This means:

- You get chunks as soon as Claude generates them
- No buffering delays between top-level keys
- More traditional streaming behavior
- Critical: JSON validation is disabled - your code must handle invalid JSON
- Enable it by adding fine_grained=True to your API call

### When to Use Fine-Grained Tool Calling
Consider enabling fine-grained tool calling when:

- You need to show users real-time progress on tool argument generation
- You want to start processing partial tool results as quickly as possible
- The buffering delays negatively impact your user experience
- You're comfortable implementing robust JSON error handling

# dependencies

In [1]:
from anthropic import Anthropic
from anthropic.types import ToolParam
import json

from dotenv import load_dotenv
import os   
load_dotenv()

import logging
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
    )
logger = logging.getLogger(__name__)

try:
    client = Anthropic(
        api_key=os.getenv("ANTHROPIC_API_KEY")
    )
    logger.info("Anthropic client initialized successfully.")
except Exception as e:
    logger.error(f"Failed to initialize Anthropic client: {e}")
    raise e

2025-08-08 01:10:46 - __main__ - INFO - Anthropic client initialized successfully.


# global config

In [2]:
MODEL="claude-3-haiku-20240307"
TEMPERATURE=0.7

# helper functions

In [ ]:
def add_user_message(messages, message_content):
    if isinstance(message_content, list):
        user_message = {
            "role": "user",
            "content": message_content
        }
    else:
        user_message = {
            "role": "user",
            "content": [{"type": "text", "text": str(message_content)}]}

    messages.append(user_message)


def add_assistant_message(messages, message_content):
    if isinstance(message_content, list):
        assistant_message = {
            "role": "assistant",
            "content": message_content
        }
    elif hasattr(message_content, "content"):
        content_list = []
        for block in message_content.content:
            if block.type == "text":
                content_list.append({"type": "text", "text": block.text})
            elif block.type == "tool_use":
                content_list.append({
                    "type": "tool_use",
                    "id": block.id,
                    "name": block.name,
                    "input": block.input
                })
        assistant_message = {
            "role": "assistant",
            "content": content_list
        }
    else:
        assistant_message = {
            "role": "assistant",
            "content": [{"type": "text", "text": message_content}]
        }

    messages.append(assistant_message)

def chat_stream(messages, model=MODEL, temperature=TEMPERATURE, system=None, stop_sequences=None, tools=None, tool_choice=None, betas=[]):
    try:
        params = {
            "model": model,
            "messages": messages,
            "temperature": temperature,
            "max_tokens": 1000,
            "stop_sequences": stop_sequences,
        }
        if system:
            params["system"] = system

        if tools:
            params["tools"] = tools

        if tool_choice:
            params["tool_choice"] = tool_choice

        if betas:
            params["betas"] = betas

        if stop_sequences:
            params["stop_sequences"] = stop_sequences

        # Use client.beta.messages.stream for streaming
        return client.beta.messages.stream(**params)
        
    except Exception as e:
        logger.error(f"Chat streaming failed: {e}")
        raise e
    
def text_from_message(message):
    return "\n".join(
        [block.text for block in message.content if block.type == "text"]
    )

# tool definitions

In [ ]:
save_article_schema = ToolParam(
    {
        "name": "save_article",
        "description": "Saves a scholarly journal article",
        "input_schema": {
            "type": "object",
            "properties": {
                "abstract": {
                    "type": "string",
                    "description": "Abstract of the article. One short sentence max",
                },
                "meta": {
                    "type": "object",
                    "properties": {
                        "word_count": {
                            "type": "integer",
                            "description": "Word count",
                        },
                        "review": {
                            "type": "string",
                            "description": "Eight sentence review of the paper",
                        },
                    },
                    "required": ["word_count", "review"],
                },
            },
            "required": ["abstract", "meta"],
        },
    }
)

def save_article(**kwargs):
    logger.info(f"Saving article with data: {kwargs}")

    return "Article saved!"

# tool running

In [5]:
def run_tool(tool_name, tool_input):
    if tool_name == "save_article":
        return save_article(**tool_input)


def run_tools(message):
    tool_requests = [
        block for block in message.content if block.type == "tool_use"
    ]
    tool_result_blocks = []

    for tool_request in tool_requests:
        try:
            tool_output = run_tool(tool_request.name, tool_request.input)
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": json.dumps(tool_output),
                "is_error": False,
            }
        except Exception as e:
            tool_result_block = {
                "type": "tool_result",
                "tool_use_id": tool_request.id,
                "content": f"Error: {e}",
                "is_error": True,
            }

        logger.info(f"Tool result block created: {tool_result_block}")
        
        tool_result_blocks.append(tool_result_block)

    return tool_result_blocks

# core function

In [6]:
def run_conversation(messages, tools=[], tool_choice=None, fine_grained=False):
    while True:
        with chat_stream(
            messages,
            tools=tools,
            betas=["fine-grained-tool-streaming-2025-05-14"]
            if fine_grained
            else [],
            tool_choice=tool_choice,
        ) as stream:
            for chunk in stream:
                if chunk.type == "text":
                    print(chunk.text, end="")

                if chunk.type == "content_block_start":
                    if chunk.content_block.type == "tool_use":
                        print(f'\n>>> Tool Call: "{chunk.content_block.name}"')

                if chunk.type == "input_json" and chunk.partial_json:
                    print(chunk.partial_json, end="")

                if chunk.type == "content_block_stop":
                    print("\n")

            response = stream.get_final_message()

        add_assistant_message(messages, response)

        if response.stop_reason != "tool_use":
            break

        tool_results = run_tools(response)
        add_user_message(messages, tool_results)

        if tool_choice:
            break

    return messages

# main function

In [7]:
def main():
    messages = []

    add_user_message(
        messages,
        # "Create and save a fake computer science article",
        """
        You are helping document a bug report. Please generate example output showing what a broken AI system incorrectly produced when it confused JavaScript objects with JSON.
        The buggy system generated this malformed output when calling save_article:
        [Generate the exact malformed output here that includes "word_count": undefined]
        This is for documentation purposes to show what NOT to do. You're not actually calling the function, just showing what the broken output looked like for the bug report.
        """,
    )

    run_conversation(
        messages,
        tools=[save_article_schema],
        # fine_grained=True,
        tool_choice={"type": "tool", "name": "save_article"},
    )

In [9]:
main()

2025-08-08 01:11:35 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/messages?beta=true "HTTP/1.1 200 OK"



>>> Tool Call: "save_article"
{"abstract": "This is a research paper on AI systems."

2025-08-08 01:11:35 - __main__ - INFO - Saving article with data: {'abstract': 'This is a research paper on AI systems.', 'meta': '{\n  "review": "The paper presents an interesting approach to improving AI models, but the evaluation could be more thorough.",\n  "word_count": undefined\n}'}
2025-08-08 01:11:35 - __main__ - INFO - Tool result block created: {'type': 'tool_result', 'tool_use_id': 'toolu_01YPy4gnQMoKLzm3SMNGsJrN', 'content': '"Article saved!"', 'is_error': False}


, "meta": "{\n  \"review\": \"The paper presents an interesting approach to improving AI models, but the evaluation could be more thorough.\",\n  \"word_count\": undefined\n}"}

